# Building a Hybrid Machine Learning Model with Python

Let’s walk through the process of building a hybrid machine learning model step by step. The idea is to combine the strengths of two distinct models to create a powerful hybrid approach. For this task, I’ll be using a specific dataset, which you can download from the provided link.

To get started, we’ll begin by importing the required Python libraries and loading the dataset. Let’s dive in!

In [139]:
import pandas as pd
data = pd.read_csv("apple_stock_data.csv")
print(data.head())

                        Date   Adj Close       Close        High         Low  \
0  2023-11-02 00:00:00+00:00  176.665985  177.570007  177.779999  175.460007   
1  2023-11-03 00:00:00+00:00  175.750671  176.649994  176.820007  173.350006   
2  2023-11-06 00:00:00+00:00  178.317520  179.229996  179.429993  176.210007   
3  2023-11-07 00:00:00+00:00  180.894333  181.820007  182.440002  178.970001   
4  2023-11-08 00:00:00+00:00  181.958893  182.889999  183.449997  181.589996   

         Open    Volume  
0  175.520004  77334800  
1  174.240005  79763700  
2  176.380005  63841300  
3  179.179993  70530000  
4  182.350006  49340300  


For the stock market dataset, I’ll convert the date column to datetime, set it as the index, and focus on the Close price.

In [140]:
print(data.dtypes[0])
import warnings
warnings.filterwarnings("ignore")

object


In [141]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data[['Close']]


In [142]:
print(data.head())
print(data.shape)



                                Close
Date                                 
2023-11-02 00:00:00+00:00  177.570007
2023-11-03 00:00:00+00:00  176.649994
2023-11-06 00:00:00+00:00  179.229996
2023-11-07 00:00:00+00:00  181.820007
2023-11-08 00:00:00+00:00  182.889999
(252, 1)


# Choosing the Hybrid Models
I’ll use LSTM for capturing sequential dependencies in stock prices and Linear Regression to model broader trends. Combining both will balance the strengths of each model for more accurate predictions.

Next, I’ll scale the Close price data between 0 and 1 using MinMaxScaler to ensure compatibility with the LSTM model.

In [143]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
data['Close'] = scaler.fit_transform(data[["Close"]])

In [144]:
print(data.iloc[0])  # Value at the first row and first column



Close    0.175853
Name: 2023-11-02 00:00:00+00:00, dtype: float64


Now, let’s prepare the data for LSTM by creating sequences of a defined length (e.g., 60 days) to predict the next day’s price:

In [145]:
import numpy as np
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(data['Close'].values, seq_length)

In [146]:
X.shape


(192, 60)

In [147]:
X

array([[0.1758535 , 0.16298258, 0.19907662, ..., 0.3836038 , 0.37395072,
        0.32232785],
       [0.16298258, 0.19907662, 0.23531069, ..., 0.37395072, 0.32232785,
        0.27140452],
       [0.19907662, 0.23531069, 0.2502798 , ..., 0.32232785, 0.27140452,
        0.30581984],
       ...,
       [0.5907946 , 0.62702868, 0.67585339, ..., 0.92907118, 0.956911  ,
        0.96068834],
       [0.62702868, 0.67585339, 0.71684399, ..., 0.956911  , 0.96068834,
        0.9107444 ],
       [0.67585339, 0.71684399, 0.73489091, ..., 0.96068834, 0.9107444 ,
        0.85212657]])

In [148]:
y.shape

(192,)

In [149]:
y

array([0.27140452, 0.30581984, 0.29169009, 0.31729147, 0.3399553 ,
       0.3414942 , 0.32624523, 0.33365987, 0.30987682, 0.28035806,
       0.26790704, 0.26385005, 0.24216562, 0.23167317, 0.24230566,
       0.27098484, 0.2451036 , 0.22607729, 0.2466425 , 0.22971459,
       0.22034137, 0.2050924 , 0.14129836, 0.07162836, 0.05763844,
       0.05595971, 0.08016223, 0.10842194, 0.11513705, 0.08575833,
       0.11191942, 0.10660318, 0.12199219, 0.15500843, 0.19124229,
       0.08911577, 0.10184666, 0.08184116, 0.06589266, 0.11625627,
       0.09065467, 0.07036932, 0.05372127, 0.06505308, 0.05344163,
       0.0640739 , 0.04826521, 0.06533294, 0.03889198, 0.14045878,
       0.16158371, 0.10758258, 0.06127595, 0.04196978, 0.02853936,
       0.        , 0.01175149, 0.02658078, 0.05623957, 0.06841074,
       0.06015673, 0.11891439, 0.07456634, 0.06015673, 0.11233911,
       0.25713495, 0.23377179, 0.24342466, 0.2481814 , 0.273783  ,
       0.25251824, 0.29770565, 0.31379398, 0.34583104, 0.34750

Now, we will split the sequences into training and test sets (e.g., 80% training, 20% testing):

In [150]:
train_size = int(len(X)*0.8)
X_train , X_test = X[:train_size] , X[train_size:]
y_train , y_test = y[:train_size] , y[train_size:]

Now, we will build a sequential LSTM model with layers to capture the temporal dependencies in the data:

In [151]:
from tensorflow.keras.models import Sequential   
from tensorflow.keras.layers import LSTM, Dense  
   
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

In [152]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.2406
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0506
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0382
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0161
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0225
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0181
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0135
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0115
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0111
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0111
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0095
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0102
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0103
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0093
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0080
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━

Now, let’s train the second model. I’ll start by generating lagged features for Linear Regression (e.g., using the past 3 days as predictors):

In [153]:
data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()
data

,Close,Lag_1,Lag_2,Lag_3
Date,,,,
2023-11-07 00:00:00+00:00,0.235311,0.199077,0.162983,0.175853
2023-11-08 00:00:00+00:00,0.250280,0.235311,0.199077,0.162983
2023-11-09 00:00:00+00:00,0.243565,0.250280,0.235311,0.199077
2023-11-10 00:00:00+00:00,0.299384,0.243565,0.250280,0.235311
2023-11-13 00:00:00+00:00,0.277001,0.299384,0.243565,0.250280
...,...,...,...,...
2024-10-28 00:00:00+00:00,0.956911,0.929071,0.917320,0.919978
2024-10-29 00:00:00+00:00,0.960688,0.956911,0.929071,0.917320
2024-10-30 00:00:00+00:00,0.910744,0.960688,0.956911,0.929071


In [154]:
len(data)

249

In [155]:
X_lin = data[['Lag_1', 'Lag_2', 'Lag_3']]
y_lin = data['Close']
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]
X_train_lin.shape

(153, 3)

Now lets train Liner Regression model


In [156]:
from sklearn.linear_model import LinearRegression
lin_model =  LinearRegression()
lin_model.fit(X_train_lin, y_train_lin)

LinearRegression()

Now, here’s how to make predictions using LSTM on the test set and inverse transform the scaled predictions:

In [157]:
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 464ms/step


In [158]:
X_test_lstm.shape

(39, 60, 1)

In [159]:
lstm_predictions

array([[223.45244],
       [222.70625],
       [222.0603 ],
       [221.52716],
       [221.09077],
       [220.44815],
       [219.70631],
       [219.11656],
       [219.06369],
       [219.3787 ],
       [219.86168],
       [220.48117],
       [221.11652],
       [221.78229],
       [222.44696],
       [223.32599],
       [223.98914],
       [224.4848 ],
       [224.77873],
       [224.96579],
       [224.82054],
       [224.63846],
       [224.62328],
       [224.70934],
       [224.7934 ],
       [225.05058],
       [225.54256],
       [226.07927],
       [226.63487],
       [227.3089 ],
       [228.10258],
       [228.90952],
       [229.43663],
       [229.72133],
       [229.85524],
       [229.96907],
       [230.07675],
       [230.00429],
       [229.604  ]], dtype=float32)

In [160]:
lin_predictions = lin_model.predict(X_test_lin)
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))

In [161]:
lin_predictions.shape

(96, 1)

Let’s see how to make predictions for the next 10 days using our hybrid model. Here’s how to predict the Next 10 Days using LSTM:

In [162]:
lstm_future_predictions = []
last_sequence = X[-1].reshape(1, seq_length, 1)
for _ in range(10):
    lstm_pred = lstm_model.predict(last_sequence)[0, 0]
    lstm_future_predictions.append(lstm_pred)
    lstm_pred_reshaped = np.array([[lstm_pred]]).reshape(1, 1, 1)
    last_sequence = np.append(last_sequence[:, 1:, :], lstm_pred_reshaped, axis=1)
lstm_future_predictions = scaler.inverse_transform(np.array(lstm_future_predictions).reshape(-1, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


Here’s how to predict the Next 10 Days using Linear Regression:

In [163]:
recent_data = data['Close'].values[-3:]
lin_future_predictions = []
for _ in range(10):
    lin_pred = lin_model.predict(recent_data.reshape(1, -1))[0]
    lin_future_predictions.append(lin_pred)
    recent_data = np.append(recent_data[1:], lin_pred)
lin_future_predictions = scaler.inverse_transform(np.array(lin_future_predictions).reshape(-1, 1))

And, here’s how to combine the predictive power of both models to make predictions for the next 10 days:

In [164]:
hybrid_future_predictions = (0.7 * lstm_future_predictions) + (0.3 * lin_future_predictions)

Here’s how to create the final DataFrame to look at the predictions:

In [165]:
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=10)
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'LSTM Predictions': lstm_future_predictions.flatten(),
    'Linear Regression Predictions': lin_future_predictions.flatten(),
    'Hybrid Model Predictions': hybrid_future_predictions.flatten()
})
print(predictions_df)

                       Date  LSTM Predictions  Linear Regression Predictions  \
0 2024-11-02 00:00:00+00:00        229.604004                     230.355192   
1 2024-11-03 00:00:00+00:00        229.163605                     225.707291   
2 2024-11-04 00:00:00+00:00        228.697357                     222.703426   
3 2024-11-05 00:00:00+00:00        228.215302                     230.631535   
4 2024-11-06 00:00:00+00:00        227.723969                     225.486380   
5 2024-11-07 00:00:00+00:00        227.227692                     222.494588   
6 2024-11-08 00:00:00+00:00        226.729324                     230.930195   
7 2024-11-09 00:00:00+00:00        226.230789                     225.245599   
8 2024-11-10 00:00:00+00:00        225.733444                     222.284007   
9 2024-11-11 00:00:00+00:00        225.238373                     231.252375   

   Hybrid Model Predictions  
0                229.829351  
1                228.126708  
2                226.899178  

# SUMMARY
In this approach, we combine multiple machine learning models to create a hybrid system that leverages the strengths of each model. A hybrid model is useful when a single algorithm can't fully capture the complexity of the data or when diverse patterns are present. By combining models like LSTM and Linear Regression, we can improve predictions by capturing both temporal dependencies and overall trends in the data.